## Import Helpers

In [0]:
from Helpers import *

sc = SparkContext.getOrCreate()
sql_context = SQLContext(spark)

## Input Parameters

In [0]:
# Input Parameters
print(f"AWS_REGION: {AWS_REGION}")
print(f"LOAD_DATE: {LOAD_DATE}")
print(f"STATUS_TABLE: {STATUS_TABLE}")
print(f"SNS_TOPIC: {SNS_TOPIC}")

print(f"SOURCE_BUCKETS_STR: {SOURCE_BUCKETS_STR}")
print(f"SOURCE_BUCKETS: {SOURCE_BUCKETS}")
print(f"RWE_BUCKET: {RWE_BUCKET}")

print(f"SUBMIT_JOB: {SUBMIT_JOB}")
print(f"PROCESSING_JOB : {PROCESSING_JOB}")

print(f"BASE_TIMEOUT : {BASE_TIMEOUT}")
print(f"PROCESSING_TIMEOUT : {PROCESSING_TIMEOUT}")

# Initialize the SuperiorProcessStatusHandler for use (will update pending_batches for each bucket)
process_status_handler = SuperiorProcessStatusHandler(AWS_REGION, STATUS_TABLE, SNS_TOPIC, LOAD_DATE)

# Initialize logger object for threading task
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

## Functions

#### CSV to PQT Function

In [0]:
"""
Function: load_data_to_s3
Description: Given a source bucket, landing bucket, input folder, key/shard/batch_id and StatusHandler object,
read in the source files, create record in the ProcessStatus table, convert to parquet and update the ProcessStatus table.
"""
def load_data_to_s3(source_bucket, landing_bucket, prefix, status_handler):
        try:
            file_location = f"s3a://{source_bucket}/"
            measurement_file = file_location + 'measurements/' + prefix
            device_file = file_location + 'devices/' + prefix
            user_file = file_location + 'users/' + prefix
            key = prefix

            measurement_csv_data = spark.read.csv(measurement_file,
                        sep=",", 
                        header="true",
                        escape="\"",
                        quote="\"",
                        schema=measurement_schema
                        )
            user_csv_data = spark.read.csv(user_file,
                        sep=",", 
                        header="true",
                        schema=user_schema
                        )

            device_csv_data = spark.read.csv(device_file,
                        sep=",", 
                        header="true",
                        schema=device_schema
                        )

            logging.info('Key for this batch: ' + key)
            logging.info('Completed reading the data for: ' + file_location + '/file_type/' + key)

            device_csv_data.createOrReplaceTempView("device_csv")

            # Filter for only valid DeviceTypeIDs     
            device_parquet = spark.sql(f"select * from device_csv where DeviceTypeID IN({','.join(str(d) for d in device_type_set)})")

            # Parquet conversion of measurement, device, & user
            parquet_user_file_location = "s3a://{}/{}/user/{}/".format(landing_bucket, LANDING_PATH, key)
            parquet_measurement_file_location = "s3a://{}/{}/measurement/{}/".format(landing_bucket, LANDING_PATH, key)
            parquet_device_file_location = "s3a://{}/{}/device/{}/".format(landing_bucket, LANDING_PATH, key)

            logging.info("Writing to parquet user " + parquet_user_file_location)

            user_csv_data.write.mode("overwrite").parquet(parquet_user_file_location)

            logging.info("Writing parquet measurement " + parquet_measurement_file_location)
            measurement_csv_data.write.mode("overwrite").parquet(parquet_measurement_file_location)

            logging.info("Writing parquet device " + parquet_device_file_location)
            device_parquet.write.mode("overwrite").parquet(parquet_device_file_location)

            # Update Process-Status table with PQT format
            logging.info(f"Updating s3 key format to pqt for {key}")
            status_handler.update_batch_format(key, fmt='pqt')
        except Exception as e:
            logging.info('Error in function: load_data_from_s3', e) 
            raise SourceMissingException(str(e))

        return f'Success: {prefix}'

## Ingestion Processing

#### Find fin files

In [0]:
def get_batch_ids(source_bucket, load_date):
    # Read the fin files on the source bucket, creating a list of batch ids using the filepath
    fin_df = spark.read.csv(f"s3://{source_bucket}/fin/{load_date}/*/*") \
        .select(col("_metadata").getField("file_path").alias("fName"))
    fin_df = fin_df.withColumn("batch_id", extract_batch_udf(col("fName")))
    batches = fin_df.select("batch_id").distinct().toPandas()['batch_id'].to_list()
    return batches

#### Populate Status Table with Pending Batches

In [0]:
def populate_pending_batches(batches, processed_batches):
    # Filter out batches that have been processed already
    unprocessed_batches = [batch for batch in batches if batch not in processed_batches]
    print(f"Batches to process are: {unprocessed_batches}")

    # Notify if there was no total batches to process in this bucket 
    # (exit condition is not on unprocessed batches to allow for retries)
    if len(batches) == 0:
        SNSNotifier(AWS_REGION, SNS_TOPIC, LOAD_DATE).notify_error(f'Submit Job Found Zero batches to Process')
        raise SourceMissingException("Submit Job found zero batches to process.")

    # Populate Process-Status Table with unprocessed batches
    process_status_handler.set_pending_batches(unprocessed_batches)
    process_status_handler.populate_pending_batches(fmt='csv') # Populate table with Pending shards to be processed

    return unprocessed_batches

In [0]:
processed_batches = process_status_handler.get_items_with_status('Processed', LOAD_DATE)
processed_batches = [batch['Key'] for batch in processed_batches]
print(f"Processed batches are: {processed_batches}")

for src_bucket in SOURCE_BUCKETS:
    print(f"Processing: {src_bucket}")
    batches = get_batch_ids(src_bucket, LOAD_DATE)
    print(f"All batches from this bucket: {batches}")
    unprocessed_batches = populate_pending_batches(batches, processed_batches)
    
    futures = []
    # 10 Threads was tested to give sufficient parallelism for this workload
    with concurrent.futures.ThreadPoolExecutor(max_workers=PARQUET_WRITE_THREADS) as executor:
        # Iterate over each bucket and prefix/batch, convert to parquet, archive files associated with shard
        for i,batch in enumerate(unprocessed_batches):
            print(f'Submit thread {i} for converting Shard (CSV -> PQT): {batch}')
            
            # Load the data for this shard
            futures.append(executor.submit(load_data_to_s3, src_bucket, RWE_BUCKET, batch, process_status_handler))
            
        concurrent.futures.wait(futures)
        results = [future.result() for future in futures]
        for res in results:
            print(res)

## Run Next Job (Processing Submit Job)

In [0]:
# Run the Submit Job notebook to continue ETL pipeline
dbutils.notebook.run(SUBMIT_JOB, BASE_TIMEOUT, dbutils.widgets.getAll())